In [27]:
import nltk
import numpy as np
import pandas as pd
import re
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [31]:
# Read data from files 
train = pd.read_csv("train.csv").fillna("")
test  = pd.read_csv("test.csv").fillna("")

### Explore the training and test date a bit

In [32]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [33]:
test.head()

,id,query,product_title,product_description
0,3,electric griddle,Star-Max 48 in Electric Griddle,
1,6,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,
2,9,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...
3,11,aveeno shampoo,AVEENO 10.5FLOZ NRSH SHINE SH,"Water, Ammonium Lauryl Sulfate, Dimethicone, S..."
4,12,flea and tick control for dogs,Merial Frontline Plus Flea and Tick Control fo...,


### My data scrubbing

In [46]:
# make a copy of the data set to scrub
jfaTrain = train

### Scrub HTML
Via BeautifulSoup 

In [57]:
import nltk
nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
print stopwords.words("english") 

showing info http://www.nltk.org/nltk_data/
[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', 

In [59]:
import re
from bs4 import BeautifulSoup

In [61]:
def description_to_words( raw_description ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    description_text = BeautifulSoup(raw_description).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", description_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 
 

### Test data scrubbing function

In [69]:
clean_description = review_to_words( train["product_description"][45] )
print "Original Description"
print "===================="
print train["product_description"][45]
print
print
print "Cleaned Description"
print "==================="
print clean_description

Original Description
Keep your MacBook Air looking clean and pristine with the MacAlly 13" MacBook Air Case. This lightweight, durable case features a 2-piece snap-on design to protect your MacBook Air from dings and scratches. It offers a custom fit while still providing access to all ports, sensors and vent openings to reduce heat build-up. This state-of-the-art 13" MacBook Case includes anti-slip silica gel feet and is designed specifically for 13" models only. It allows you to transport your computer without worry.


Cleaned Description
keep macbook air looking clean pristine macally macbook air case lightweight durable case features piece snap design protect macbook air dings scratches offers custom fit still providing access ports sensors vent openings reduce heat build state art macbook case includes anti slip silica gel feet designed specifically models allows transport computer without worry


### Apply data scrubbing function to data

In [71]:
num_descriptions         = train["product_description"].size
clean_train_descriptions = []
for i in xrange( 0, num_descriptions ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Description %d of %d\n" % ( i+1, num_descriptions )                                                                    
    clean_train_descriptions.append( description_to_words_to_words( train["product_description"][i] ))

Description 1000 of 10158

Description 2000 of 10158

Description 3000 of 10158


/Users/jason/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/jason/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP cl


Description 4000 of 10158

Description 5000 of 10158

Description 6000 of 10158

Description 7000 of 10158

Description 8000 of 10158

Description 9000 of 10158

Description 10000 of 10158



/Users/jason/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/6552101" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


In [81]:
num_titles         = train["product_title"].size
clean_train_titles = []
for i in xrange( 0, num_titles ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Title %d of %d\n" % ( i+1, num_titles )                                                                    
    clean_train_titles.append( description_to_words( train["product_title"][i] ))

Title 1000 of 10158

Title 2000 of 10158

Title 3000 of 10158

Title 4000 of 10158

Title 5000 of 10158

Title 6000 of 10158

Title 7000 of 10158

Title 8000 of 10158

Title 9000 of 10158

Title 10000 of 10158



In [113]:
num_queries         = train["query"].size
clean_train_queries = []
for i in xrange( 0, num_queries ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Description %d of %d..." % ( i+1, num_queries )                                                                    
    clean_train_queries.append( description_to_words( train["query"][i] ))

Description 1000 of 10158...
Description 2000 of 10158...
Description 3000 of 10158...
Description 4000 of 10158...
Description 5000 of 10158...
Description 6000 of 10158...
Description 7000 of 10158...
Description 8000 of 10158...
Description 9000 of 10158...
Description 10000 of 10158...


In [82]:
train["product_description"] = clean_train_descriptions
train["product_title"]       = clean_train_titles
train["query"]               = clean_train_queries

### Create Feature mapper class

In [83]:
class FeatureMapper:
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        for feature_name, column_name, extractor in self.features:
            extractor.fit(X[column_name], y)

    def transform(self, X):
        extracted = []
        for feature_name, column_name, extractor in self.features:
            fea = extractor.transform(X[column_name])
            if hasattr(fea, "toarray"):
                extracted.append(fea.toarray())
            else:
                extracted.append(fea)
        if len(extracted) > 1:
            return np.concatenate(extracted, axis=1)
        else: 
            return extracted[0]

    def fit_transform(self, X, y=None):
        extracted = []
        for feature_name, column_name, extractor in self.features:
            fea = extractor.fit_transform(X[column_name], y)
            if hasattr(fea, "toarray"):
                extracted.append(fea.toarray())
            else:
                extracted.append(fea)
        if len(extracted) > 1:
            return np.concatenate(extracted, axis=1)
        else: 
            return extracted[0]

def identity(x):
    return x

In [84]:
class SimpleTransform(BaseEstimator):
    def __init__(self, transformer=identity):
        self.transformer = transformer

    def fit(self, X, y=None):
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X)

    def transform(self, X, y=None):
        return np.array([self.transformer(x) for x in X], ndmin=2).T

#                          Feature Set Name            Data Frame Column              Transformer
features = FeatureMapper([('QueryBagOfWords',          'query',                       CountVectorizer(max_features=200)),
                  

SyntaxError: invalid syntax (<ipython-input-84-4b1844b9838b>, line 16)

### Instantiate FeatureMapper

In [85]:
#                          Feature Set Name            Data Frame Column              Transformer
features = FeatureMapper([('QueryBagOfWords',          'query',                       CountVectorizer(max_features=200)),
                          ('TitleBagOfWords',          'product_title',               CountVectorizer(max_features=200)),
                          ('DescriptionBagOfWords',    'product_description',         CountVectorizer(max_features=200)),
                          ('QueryTokensInTitle',       'query_tokens_in_title',       SimpleTransform()),
                          ('QueryTokensInDescription', 'query_tokens_in_description', SimpleTransform())])

In [ ]:
data = train
token_pattern = re.compile(r"(?u)\b\w\w+\b")
data["query_tokens_in_title"] = 0.0
data["query_tokens_in_description"] = 0.0
data.head()

In [138]:
row = data.loc[1,:]

In [139]:
query       = set(x.lower() for x in token_pattern.findall(row["query"]))
title       = set(x.lower() for x in token_pattern.findall(row["product_title"]))
description = set(x.lower() for x in token_pattern.findall(row["product_description"]))


In [162]:
len(query.intersection(title)) / len(title)

foo = len(query.intersection(title))
bar = len(title)

print foo
print bar
print foo / bar

foo = 3
bar = 10

print foo
print bar
print float(foo) / float(bar)
print float(foo/bar)

3
10
0
3
10
0.3
0.0


In [163]:
def extract_features(data):
    token_pattern = re.compile(r"(?u)\b\w\w+\b")
    data["query_tokens_in_title"] = 0.0
    data["query_tokens_in_description"] = 0.0
    for i, row in data.iterrows():
        query       = set(x.lower() for x in token_pattern.findall(row["query"]))
        title       = set(x.lower() for x in token_pattern.findall(row["product_title"]))
        description = set(x.lower() for x in token_pattern.findall(row["product_description"]))
        if len(title) > 0:
            data.set_value(i, "query_tokens_in_title", float(len(query.intersection(title)))/float(len(title)))
        if len(description) > 0:
            data.set_value(i, "query_tokens_in_description", float(len(query.intersection(description)))/float(len(description)))

extract_features(train)
extract_features(test)

In [164]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance,query_tokens_in_title,query_tokens_in_description
0,1,bridal shower decorations,accent pillow heart design red black,red satin accent pillow embroidered heart blac...,1,0.000,0.000000,0.000000
1,2,led christmas lights,set battery operated multi led train christmas...,set battery operated train christmas lights it...,4,0.000,0.300000,0.037037
2,4,projector,viewsonic pro dlp multimedia projector,,4,0.471,0.200000,0.000000
3,5,wine rack,concept housewares wr solid wood ceiling wall ...,like silent sturdy tree southern enterprises b...,4,0.000,0.153846,0.016949
4,7,light bulb,wintergreen lighting christmas led light bulb ...,wtgr features nickel base average hours acryli...,2,0.471,0.285714,0.062500


### Pass into Classifier

In [165]:
pipeline = Pipeline([("extract_features", features),
                     ("classify", RandomForestClassifier(n_estimators=200,
                                                         n_jobs=1,
                                                         min_samples_split=2,
                                                         random_state=1))])

pipeline.fit(train, train["median_relevance"])

predictions = pipeline.predict(test)

In [166]:
submission = pd.DataFrame({"id": test["id"], "prediction": predictions})
submission.to_csv("python_benchmark.csv", index=False)

# Create a CROSS-VALIDATION script

In [93]:
from sklearn import metrics
from sklearn import cross_validation

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings

def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

# Kappa Scorer 
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)

In [94]:
?cross_validation.StratifiedKFold

In [111]:
cv = cross_validation.StratifiedKFold(len(train), n_folds=5)

TypeError: object of type 'int' has no len()

In [97]:
?cross_validation.cross_val_score

In [112]:
cross_validation.cross_val_score(pipeline, train)

ValueError: Expected array-like (array or non-string sequence), got None

In [101]:
train.shape

(10158, 8)

In [102]:
test.shape

(22513, 6)

NameError: name 'size' is not defined